In [1]:
import pandas as pd
import numpy as np
import math
import scipy
from scipy.stats import norm

### **Trade quantities**
* Sell $N_p = 1000$ put contracts
* Buy $N_c$ call contracts (paid for using put premium)
* Cash leftover: $C_0$ from strategy

### **Underlying**
* SPY price:
  * At initiation: $S_0$
  * One week before expiry: $S_t$

### **Option values**
* Put:
  * At initiation: $V_{p,0}$
  * At time $t$: $V_{p,t}$
* Call:
  * At initiation: $V_{c,0}$
  * At time $t$: $V_{c,t}$

---

## **Premium-Financing Constraint**

5 % OTM Put-sale premium finances 5 % OTM calls plus leftover cash:

$$1000 \cdot V_{p,0} = N_c \cdot V_{c,0} + C_0$$

Hence:

$$N_c = \frac{1000 \cdot V_{p,0} - C_0}{V_{c,0}}$$

---

## **P&L One Week Before Expiry**

Total mark-to-market P&L at time $t$:

$$\text{PNL}_t = \underbrace{1000 \cdot (V_{p,0} - V_{p,t})}_{\text{Short Put P\&L}} + \underbrace{N_c \cdot (V_{c,t} - V_{c,0})}_{\text{Long Call P\&L}} + \underbrace{C_0(e^{rt} - 1)}_{\text{Interest on Cash}}$$

Substitute $N_c = \frac{1000 \cdot V_{p,0} - C_0}{V_{c,0}}$:

$$\boxed{\text{PNL}_t = 1000 \cdot (V_{p,0} - V_{p,t}) + \frac{1000 \cdot V_{p,0} - C_0}{V_{c,0}} \cdot (V_{c,t} - V_{c,0}) + C_0(e^{rt} - 1)}$$

---

## **Interpretation**

* If the 95% put decays or SPY rises → **positive** put P&L
* If the 105% call gains value → **positive** call P&L
* Cash earns risk-free interest at rate $r$ over time $t$
* Combined structure ≈ **short vol + long convexity financed using premium + interest on residual cash**

In [2]:
class OptionsPricingModel:
    """Black-Scholes-Merton model for SPY options pricing"""
    
    def __init__(self, spot_price, risk_free_rate, dividend_yield):
        self.S0 = spot_price
        self.r = risk_free_rate
        self.q = dividend_yield  # SPY dividend yield
        
    def black_scholes_price(self, K, T, sigma, option_type='call'):
        """
        Calculate Black-Scholes price for European options
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Implied volatility
        option_type: 'call' or 'put'
        """
        d1 = (np.log(self.S0 / K) + (self.r - self.q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if option_type == 'call':
            price = self.S0 * np.exp(-self.q * T) * norm.cdf(d1) - K * np.exp(-self.r * T) * norm.cdf(d2)
        else:  # put
            price = K * np.exp(-self.r * T) * norm.cdf(-d2) - self.S0 * np.exp(-self.q * T) * norm.cdf(-d1)
        
        return price
    
    
    def crr_binomial_price(self, K, T, sigma, option_type='call', steps=1000, american=True):
        """
        Calculate option price using Cox-Ross-Rubinstein binomial tree
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Volatility
        option_type: 'call' or 'put'
        steps: Number of time steps in the binomial tree
        american: True for American options, False for European
        """
        dt = T / steps  # Time increment
        
        # CRR parameters
        u = np.exp(sigma * np.sqrt(dt))  # Up factor
        d = 1 / u  # Down factor
        p = (np.exp((self.r - self.q) * dt) - d) / (u - d)  # Risk-neutral probability
        discount = np.exp(-self.r * dt)  # Discount factor per step
        
        # Initialize asset prices at maturity
        ST = np.zeros(steps + 1)
        for i in range(steps + 1):
            ST[i] = self.S0 * (u ** (steps - i)) * (d ** i)
        
        # Initialize option values at maturity
        if option_type == 'call':
            option_values = np.maximum(ST - K, 0)
        else:  # put
            option_values = np.maximum(K - ST, 0)
        
        # Backward induction through the tree
        for step in range(steps - 1, -1, -1):
            for i in range(step + 1):
                # Current stock price at this node
                S = self.S0 * (u ** (step - i)) * (d ** i)
                
                # Continuation value (discounted expected value)
                continuation_value = discount * (p * option_values[i] + (1 - p) * option_values[i + 1])
                
                # Early exercise value
                if option_type == 'call':
                    exercise_value = max(S - K, 0)
                else:  # put
                    exercise_value = max(K - S, 0)
                
                # For American options, take the maximum
                if american:
                    option_values[i] = max(continuation_value, exercise_value)
                else:
                    option_values[i] = continuation_value
        
        return option_values[0]

In [3]:
from dataclasses import dataclass

@dataclass
class TradeInputs:
    # Market / model inputs
    spot_price: float
    risk_free_rate: float
    dividend_yield: float
    time_to_maturity: float  # in years

    # Put (SELL) 95% moneyness
    put_strike: float
    put_contracts: int
    put_iv: float

    # Call (BUY) 105% moneyness
    call_strike: float
    call_iv: float

    # Binomial settings
    binomial_steps: int
    american: bool

    # Which model to use for trade sizing: "bs" or "binomial"
    pricing_model: str  # e.g. "bs" or "binomial"


class BullishRiskReversalInitialSetUpAnalyzer:
    """
    Encapsulates the full pricing + report logic
    for the short put / long call trade.
    """

    def __init__(self, inputs: TradeInputs):
        self.inputs = inputs
        # Initialize the underlying pricing model
        self.model = OptionsPricingModel(
            spot_price=inputs.spot_price,
            risk_free_rate=inputs.risk_free_rate,
            dividend_yield=inputs.dividend_yield,
        )

    def compute(self) -> dict:
        """
        Run all pricing steps and return a dictionary of results
        (no printing here – just pure calculation).
        """
        i = self.inputs

        # === Step 4: Price the options (Black–Scholes) ===
        put_price_bs = self.model.black_scholes_price(
            K=i.put_strike,
            T=i.time_to_maturity,
            sigma=i.put_iv,
            option_type='put'
        )

        call_price_bs = self.model.black_scholes_price(
            K=i.call_strike,
            T=i.time_to_maturity,
            sigma=i.call_iv,
            option_type='call'
        )

        # === Step 4b: Price the options (CRR Binomial) ===
        put_price_binom = self.model.crr_binomial_price(
            K=i.put_strike,
            T=i.time_to_maturity,
            sigma=i.put_iv,
            option_type='put',
            steps=i.binomial_steps,
            american=i.american
        )

        call_price_binom = self.model.crr_binomial_price(
            K=i.call_strike,
            T=i.time_to_maturity,
            sigma=i.call_iv,
            option_type='call',
            steps=i.binomial_steps,
            american=i.american
        )

        # === Choose which model to use for trade sizing ===
        pm = i.pricing_model.lower()
        if pm == "bs":
            put_price_for_trade = put_price_bs
            call_price_for_trade = call_price_bs
        elif pm == "binomial":
            put_price_for_trade = put_price_binom
            call_price_for_trade = call_price_binom
        else:
            raise ValueError(f"Unknown pricing_model: {i.pricing_model}. Use 'bs' or 'binomial'.")

        # === Step 5: Trade economics (using chosen model for sizing) ===

        # Premium collected from selling puts
        premium_collected = put_price_for_trade * i.put_contracts * 100

        # Max number of call contracts that can afforded with that premium
        if call_price_for_trade > 0:
            call_contracts = int(premium_collected // (call_price_for_trade * 100))
        else:
            call_contracts = 0

        # Premium paid for calls using that size
        premium_paid = call_price_for_trade * call_contracts * 100

        net_premium = premium_collected - premium_paid

        return {
            "spot_price": i.spot_price,
            "risk_free_rate": i.risk_free_rate,
            "dividend_yield": i.dividend_yield,
            "time_to_maturity": i.time_to_maturity,
            "binomial_steps": i.binomial_steps,

            "pricing_model": pm,

            "put_strike": i.put_strike,
            "put_iv": i.put_iv,
            "put_contracts": i.put_contracts,
            "put_price_bs": put_price_bs,
            "put_price_binom": put_price_binom,

            "call_strike": i.call_strike,
            "call_iv": i.call_iv,
            "call_price_bs": call_price_bs,
            "call_price_binom": call_price_binom,

            # Trade economics (based on chosen model)
            "premium_collected": premium_collected,
            "call_contracts": call_contracts,
            "premium_paid": premium_paid,
            "net_premium": net_premium,
        }

    def print_report(self) -> None:
        """
        Pretty-print the same info the script printed,
        using the results from compute().
        """
        r = self.compute()

        print("=== TRADE SETUP ===")
        print(f"Spot Price: ${r['spot_price']:.2f}")
        print(f"Risk-free rate: {r['risk_free_rate']*100:.2f}%")
        print(f"Dividend yield: {r['dividend_yield']*100:.2f}%")
        print(f"Time to maturity: {r['time_to_maturity']:.4f} years")
        print(f"Binomial steps: {r['binomial_steps']}")
        print(f"Pricing model used for sizing: {r['pricing_model'].upper()}")

        print("\n--- PUT (SELL) 95% Moneyness ---")
        print(f"Strike: ${r['put_strike']:.2f}")
        print(f"Implied Vol: {r['put_iv']*100:.2f}%")
        print(f"Contracts: {r['put_contracts']}")
        print(f"  BS price per option:       ${r['put_price_bs']:.4f}")
        print(f"  Binomial price per option: ${r['put_price_binom']:.4f}")
        print(f"  Premium collected ({r['pricing_model'].upper()}): ${r['premium_collected']:,.2f}")

        print("\n--- CALL (BUY) 105% Moneyness ---")
        print(f"Strike: ${r['call_strike']:.2f}")
        print(f"Implied Vol: {r['call_iv']*100:.2f}%")
        print(f"Contracts bought ({r['pricing_model'].upper()} sizing): {r['call_contracts']}")
        print(f"  BS price per option:       ${r['call_price_bs']:.4f}")
        print(f"  Binomial price per option: ${r['call_price_binom']:.4f}")
        print(f"  Premium paid ({r['pricing_model'].upper()}): ${r['premium_paid']:,.2f}")

        print("\n=== NET PREMIUM (based on chosen model) ===")
        print(f"Net Premium: ${r['net_premium']:,.2f}")

        print("\n=== MODEL CHECK ===")
        print(
            "Put:  BS vs Binomial =",
            round(r['put_price_bs'], 4), "vs", round(r['put_price_binom'], 4)
        )
        print(
            "Call: BS vs Binomial =",
            round(r['call_price_bs'], 4), "vs", round(r['call_price_binom'], 4)
        )


In [4]:
def compute_trade_pnl(
    n_put_contracts: int,
    n_call_contracts: int,
    V_p_0: float,
    V_p_t: float,
    V_c_0: float,
    V_c_t: float,
    net_premium: float,        # C0
    risk_free_rate: float,     # r
    rf_compunding_period: float    # t in years
) -> float:
    """
    Compute the PNL of the risk reversal,

    Formula used:
        PNL_t =
            + n_put_contracts  * 100 * (V_p_0 - V_p_t)
            + n_call_contracts * 100 * (V_c_t - V_c_0)
            + net_premium * (e^(r*t) - 1)

    Where:
        n_put_contracts  : number of short put option contracts
        n_call_contracts : number of long call option contracts
        V_p_0, V_p_t     : initial and final put prices
        V_c_0, V_c_t     : initial and final call prices
        net_premium      : net premium (cash flow at t=0)
        risk_free_rate   : r
        rf_compunding_period : t in years

    Returns: pnl_total : Total Pnl of the strategy
    """

    # --- PNL from short puts ---
    term_put = n_put_contracts * 100 * (V_p_0 - V_p_t)

    # --- PNL from long calls ---
    term_call = n_call_contracts * 100 * (V_c_t - V_c_0)

    # --- Carry on cash premium ---
    term_carry = net_premium * (math.exp(risk_free_rate * rf_compunding_period) - 1)

    pnl_total = term_put + term_call + term_carry

    return pnl_total



def risk_reversal_greeks(
    n_put,
    n_call,
    put_delta,
    call_delta,
    put_gamma,
    call_gamma,
    put_vega,
    call_vega,
    put_theta,
    call_theta,
    put_rho,
    call_rho,
    contract_size=100,
):
    """
    Compute portfolio Greeks for a risk reversal.

    n_put, n_call : number of contracts (positive = long, negative = short)
    Greeks are per-option (not per-contract).
    contract_size : usually 100 for US equity options.
    """

    # Each Greek = sum(contracts * contract_size * per-option greek)
    delta = contract_size * (n_call * call_delta + n_put * put_delta)
    gamma = contract_size * (n_call * call_gamma + n_put * put_gamma)
    vega  = contract_size * (n_call * call_vega  + n_put * put_vega)
    theta = contract_size * (n_call * call_theta + n_put * put_theta)
    rho   = contract_size * (n_call * call_rho   + n_put * put_rho)

    return {
        "delta": delta,
        "gamma": gamma,
        "vega": vega,
        "theta": theta,
        "rho": rho,
    }


In [5]:
five_percent_otm_options_path = '../data/Option/eod_option_5%_OTM.csv'
df = pd.read_csv(five_percent_otm_options_path)

call = df[df["call_put"] == "C"]
spot_price = call["underlying_price"].iloc[0]

call_iv = call["iv"].iloc[0]
call_strike = call["price_strike"].iloc[0]
call_delta = call["delta"].iloc[0]
call_gamma = call["gamma"].iloc[0]
call_vega = call["vega"].iloc[0]
call_theta = call["theta"].iloc[0]
call_rho = call["rho"].iloc[0]

put = df[df["call_put"] == "P"]
put_iv = put["iv"].iloc[0]
put_strike = put["price_strike"].iloc[0]
put_delta = put["delta"].iloc[0]
put_gamma = put["gamma"].iloc[0]
put_vega = put["vega"].iloc[0]
put_theta = put["theta"].iloc[0]
put_rho = put["rho"].iloc[0]

print(f"Spot Price: {spot_price}")

print("\n--- CALL ---")
print(f"Call IV: {call_iv}")
print(f"Call Strike: {call_strike}")
print(f"Call Delta: {call_delta}")
print(f"Call Gamma: {call_gamma}")
print(f"Call Vega: {call_vega}")
print(f"Call Theta: {call_theta}")
print(f"Call Rho: {call_rho}")

print("\n--- PUT ---")
print(f"Put IV: {put_iv}")
print(f"Put Strike: {put_strike}")
print(f"Put Delta: {put_delta}")
print(f"Put Gamma: {put_gamma}")
print(f"Put Vega: {put_vega}")
print(f"Put Theta: {put_theta}")
print(f"Put Rho: {put_rho}")

Spot Price: 543.37

--- CALL ---
Call IV: 0.210945
Call Strike: 571.0
Call Delta: 0.228211
Call Gamma: 0.009197
Call Vega: 0.470783
Call Theta: -0.176452
Call Rho: 0.098448

--- PUT ---
Put IV: 0.323284
Put Strike: 516.0
Put Delta: -0.264147
Put Gamma: 0.006544
Put Vega: 0.51151
Put Theta: -0.262369
Put Rho: -0.111529


In [6]:
inputs = TradeInputs(
    spot_price=spot_price,
    risk_free_rate=0.0433,
    dividend_yield=0.0127,
    time_to_maturity=30/365,

    put_strike=put_strike,
    put_contracts=1000,
    put_iv=put_iv,

    call_strike=call_strike,
    call_iv=call_iv,

    binomial_steps=1000,
    american=True,
    pricing_model="binomial"
)

analyzer = BullishRiskReversalInitialSetUpAnalyzer(inputs=inputs)
analyzer.print_report()
initial_set_up = analyzer.compute()


=== TRADE SETUP ===
Spot Price: $543.37
Risk-free rate: 4.33%
Dividend yield: 1.27%
Time to maturity: 0.0822 years
Binomial steps: 1000
Pricing model used for sizing: BINOMIAL

--- PUT (SELL) 95% Moneyness ---
Strike: $516.00
Implied Vol: 32.33%
Contracts: 1000
  BS price per option:       $8.4602
  Binomial price per option: $8.4911
  Premium collected (BINOMIAL): $849,108.43

--- CALL (BUY) 105% Moneyness ---
Strike: $571.00
Implied Vol: 21.09%
Contracts bought (BINOMIAL sizing): 2025
  BS price per option:       $4.1938
  Binomial price per option: $4.1914
  Premium paid (BINOMIAL): $848,756.05

=== NET PREMIUM (based on chosen model) ===
Net Premium: $352.38

=== MODEL CHECK ===
Put:  BS vs Binomial = 8.4602 vs 8.4911
Call: BS vs Binomial = 4.1938 vs 4.1914


In [7]:
initial_set_up

{'spot_price': np.float64(543.37),
 'risk_free_rate': 0.0433,
 'dividend_yield': 0.0127,
 'time_to_maturity': 0.0821917808219178,
 'binomial_steps': 1000,
 'pricing_model': 'binomial',
 'put_strike': np.float64(516.0),
 'put_iv': np.float64(0.323284),
 'put_contracts': 1000,
 'put_price_bs': np.float64(8.460236074876917),
 'put_price_binom': np.float64(8.491084290464878),
 'call_strike': np.float64(571.0),
 'call_iv': np.float64(0.210945),
 'call_price_bs': np.float64(4.193847130448077),
 'call_price_binom': np.float64(4.191387904323273),
 'premium_collected': np.float64(849108.4290464878),
 'call_contracts': 2025,
 'premium_paid': np.float64(848756.0506254628),
 'net_premium': np.float64(352.37842102500144)}

In [8]:
risk_reversal_greeks(
    initial_set_up['put_contracts'],
    initial_set_up['call_contracts'],
    put_delta,
    call_delta,
    put_gamma,
    call_gamma,
    put_vega,
    call_vega,
    put_theta,
    call_theta,
    put_rho,
    call_rho,
    contract_size=100,
)

{'delta': np.float64(19798.027499999997),
 'gamma': np.float64(2516.7925),
 'vega': np.float64(146484.5575),
 'theta': np.float64(-61968.43),
 'rho': np.float64(8782.819999999998)}

### **Key Observations from Portfolio Greeks**

Following the historic rally, the trader initiated a bullish risk-reversal position—short 95% puts and long 105% calls—and the resulting Greeks reveal an aggressively leveraged, high-convexity bullish profile that carries significant risk.

**Delta: +19,569.81**
The portfolio exhibits an extremely strong positive delta, implying substantial bullish exposure. This level of delta suggests that the long call position, funded by premium collected from the short puts, is very large (~2000 contracts). As a result, the portfolio gains roughly **$19,570** for every 1-point increase in SPY. The trader is essentially holding a large synthetic long equity position.

**Gamma: +2,507.59**
The positive gamma indicates that the portfolio’s delta will increase further as SPY rises, providing upside acceleration. This convexity is characteristic of being net long options, meaning the long calls overwhelmingly dominate the profile. However, on the downside, although gamma reduces the delta as SPY falls, the trader still carries substantial exposure through the large short put position.

**Vega: +146,013.77**
The portfolio is massively long volatility. A 1-point drop in implied volatility results in approximately **$146,013** of losses. Entering the trade after a major rally is particularly dangerous, as implied volatility typically collapses following such events. This creates a scenario where the trader is exposed to significant mark-to-market losses even if the underlying moves upward.

**Theta: –61,791.98**
Time decay is severe: the trader loses about **$61,800 per day** from theta alone. This reflects the net-long-premium structure—long expensive OTM calls and short relatively cheap puts. With only a month until expiration, this decay will accelerate, requiring the underlying to move sharply and quickly to offset the decay.

**Rho: +8,684.37**
The position has moderately positive rho, benefiting from rising interest rates. While this is far less impactful than the delta, vega, and theta exposures, it still contributes marginally to the overall profile.

---

### **Critical Risks Identified**

1. **Volatility Crush Risk**
   Because the position is so heavily long volatility, any post-rally volatility collapse could cause substantial losses. This is especially concerning given the historical tendency for implied volatility to compress sharply after large upward moves.

2. **Severe Theta Bleed**
   The portfolio is losing more than $61,000 per day due to time decay. Given the short time to expiration, the trader faces rapidly accelerating theta losses unless SPY rallies hard and fast.

3. **Timing Risk**
   The trader entered after a massive rally traditionally the worst time to purchase upside calls:

   * Calls are inflated due to demand and elevated skew dynamics.
   * Puts have already seen their fear premium eroded.
   * Markets often consolidate or retrace after major rallies.

4. **Asymmetric Downside Risk**
   While the upside is theoretically unlimited, the short puts introduce substantial downside risk. Any sharp reversal could lead to significant losses well before the calls regain value.

---

### **Conclusion**

Overall, the trader has constructed an aggressively bullish position at a moment when the risk–reward profile is unfavorable. The extreme vega and theta exposure means that without a **rapid and continued upside breakout**, the position is vulnerable to large mark-to-market losses. Even a benign sideways market or a modest volatility contraction could turn the trade sharply negative.


# **Scenario Analysis for 1-mo SPY 95% Put Short vs. 105% Call Long**

### **Trade Structure (as of 4/9/2025 afternoon)**

* **Sell 1000× 1-month 95% moneyness SPY puts**

  * Initial IV (5% OTM put): **0.323**
* **Buy 1000× 1-month 105% moneyness SPY calls (using the premium)**

  * Initial IV (5% OTM call): **0.211**

### **Objective**

Project **1-week-before-expiry P&L**, under:

1. **Spot moves (over 3 weeks):**
   +10%, +5%, +2.5%, 0%, −2.5%, −5%, −10%
2. **Implied volatility adjustments:**
   Need multiple reasonable IV paths for both the **OTM call** and **OTM put**.

---

# **IV Change Assumptions**

A 3-week move toward expiry typically produces:

* **Volatility–spot correlation:** For equities, **negative**.

  * Spot up → IVs fall (especially downside puts).
  * Spot down → IVs rise (especially downside puts).
* **Skew dynamics:**

  * Downside IV rises *more* than upside IV.
  * Upside IV is relatively stable unless there is a large rally.
* **Time-to-expiry shrink:**

  * Skew becomes steeper near expiry because crash-risk premiums concentrate in short tenors.

Because of this, IV changes can be represented in **three regimes**:

1. **Spot Up → IV Softening**
2. **Spot Flat → Slight Normalization**
3. **Spot Down → IV Expansion (skew steepening)**

I provide **two sets of IV-change assumptions for each spot move**:

* **Set A: Moderate volatility reaction** (stable market conditions).
* **Set B: Strong volatility reaction** (macro stress or catalysts).

---

# **Scenario Grid (IV % Changes)**

All IV changes are relative to the initial IVs:

* **Call IV₀ = 21.1%**
* **Put  IV₀ = 32.3%**

Values are expressed as **percentage change of implied volatility**, not absolute points.

---

| **Spot Change**    | **Scenario**    | **ΔIV (105% Call)** | **ΔIV (95% Put)** | **Short Market Reasoning**                                                     |
| ------------------ | --------------- | ------------------- | ----------------- | ------------------------------------------------------------------------------ |
| **+10%**           | **A: Moderate** | −8%                 | −20%              | Strong risk-on rally, crash premium collapses; skew flattens.                  |
|                    | **B: Strong**   | −12%                | −30%              | Vol supply + short-covering; downside vol crushed aggressively.                |
| **+5%**            | **A: Moderate** | −5%                 | −12%              | Gradual rally reduces hedging demand; skew eases.                              |
|                    | **B: Strong**   | −8%                 | −18%              | Systematic vol sellers compress surface as market stabilizes.                  |
| **+2.5%**          | **A: Moderate** | −2%                 | −5%               | Mild spot drift up → softening of fear premium.                                |
|                    | **B: Strong**   | −4%                 | −8%               | Slow grind up leads to steady skew compression and lower vol-of-vol.           |
| **0% (unchanged)** | **A: Moderate** | −1%                 | −1%               | Theta decay + IV roll-down; surface normalizes slightly.                       |
|                    | **B: Strong**   | −3%                 | −2%               | More aggressive term-structure roll-down + short vol flows.                    |
| **−2.5%**          | **A: Moderate** | +1%                 | +5%               | Light hedging demand increases put premium; skew steepens.                     |
|                    | **B: Strong**   | +3%                 | +10%              | Hedge rebalancing + vol-of-vol uptick on small drawdown.                       |
| **−5%**            | **A: Moderate** | +5%                 | +15%              | Larger drop triggers meaningful put demand and skew rise.                      |
|                    | **B: Strong**   | +8%                 | +25%              | Risk-off flows → vol buyers, CTA deleveraging, skew spikes.                    |
| **−10%**           | **A: Moderate** | +10%                | +35%              | Sharp selloff → crash-risk premium surges.                                     |
|                    | **B: Strong**   | +15%                | +55%              | Panic protection buying; downside IV explosion reminiscent of crisis dynamics. |


In [9]:
iv_scenarios = {
    0.10: {
        "moderate": {"call_iv_change": -0.08, "put_iv_change": -0.20},
        "strong":   {"call_iv_change": -0.12, "put_iv_change": -0.30}
    },
    0.05: {
        "moderate": {"call_iv_change": -0.05, "put_iv_change": -0.12},
        "strong":   {"call_iv_change": -0.08, "put_iv_change": -0.18}
    },
    0.025: {
        "moderate": {"call_iv_change": -0.02, "put_iv_change": -0.05},
        "strong":   {"call_iv_change": -0.04, "put_iv_change": -0.08}
    },
    0: {
        "moderate": {"call_iv_change": -0.01, "put_iv_change": -0.01},
        "strong":   {"call_iv_change": -0.03, "put_iv_change": -0.02}
    },
    -0.025: {
        "moderate": {"call_iv_change": +0.01, "put_iv_change": +0.05},
        "strong":   {"call_iv_change": +0.03, "put_iv_change": +0.10}
    },
    -0.05: {
        "moderate": {"call_iv_change": +0.05, "put_iv_change": +0.15},
        "strong":   {"call_iv_change": +0.08, "put_iv_change": +0.25}
    },
    -0.10: {
        "moderate": {"call_iv_change": +0.10, "put_iv_change": +0.35},
        "strong":   {"call_iv_change": +0.15, "put_iv_change": +0.55}
    }
}

In [10]:
inputs = TradeInputs(
    spot_price=spot_price,
    risk_free_rate=0.0433,
    dividend_yield=0.0127,
    time_to_maturity=30/365,

    put_strike=put_strike,
    put_contracts=1000,
    put_iv=put_iv,

    call_strike=call_strike,
    call_iv=call_iv,

    binomial_steps=1000,
    american=True,
    pricing_model="binomial"
)

analyzer = BullishRiskReversalInitialSetUpAnalyzer(inputs=inputs)
analyzer.print_report()      
initial_set_up = analyzer.compute()

initial_set_up 

=== TRADE SETUP ===
Spot Price: $543.37
Risk-free rate: 4.33%
Dividend yield: 1.27%
Time to maturity: 0.0822 years
Binomial steps: 1000
Pricing model used for sizing: BINOMIAL

--- PUT (SELL) 95% Moneyness ---
Strike: $516.00
Implied Vol: 32.33%
Contracts: 1000
  BS price per option:       $8.4602
  Binomial price per option: $8.4911
  Premium collected (BINOMIAL): $849,108.43

--- CALL (BUY) 105% Moneyness ---
Strike: $571.00
Implied Vol: 21.09%
Contracts bought (BINOMIAL sizing): 2025
  BS price per option:       $4.1938
  Binomial price per option: $4.1914
  Premium paid (BINOMIAL): $848,756.05

=== NET PREMIUM (based on chosen model) ===
Net Premium: $352.38

=== MODEL CHECK ===
Put:  BS vs Binomial = 8.4602 vs 8.4911
Call: BS vs Binomial = 4.1938 vs 4.1914


{'spot_price': np.float64(543.37),
 'risk_free_rate': 0.0433,
 'dividend_yield': 0.0127,
 'time_to_maturity': 0.0821917808219178,
 'binomial_steps': 1000,
 'pricing_model': 'binomial',
 'put_strike': np.float64(516.0),
 'put_iv': np.float64(0.323284),
 'put_contracts': 1000,
 'put_price_bs': np.float64(8.460236074876917),
 'put_price_binom': np.float64(8.491084290464878),
 'call_strike': np.float64(571.0),
 'call_iv': np.float64(0.210945),
 'call_price_bs': np.float64(4.193847130448077),
 'call_price_binom': np.float64(4.191387904323273),
 'premium_collected': np.float64(849108.4290464878),
 'call_contracts': 2025,
 'premium_paid': np.float64(848756.0506254628),
 'net_premium': np.float64(352.37842102500144)}

In [11]:
def scenario_simulation(
    initial_set_up: dict,
    iv_scenarios: dict,
    model: type[OptionsPricingModel],
    pricing_model_used: str,
    time_to_maturity: float,
    rf_compunding_period: float
) -> dict:
    """
    Run spot + IV scenarios and return, for each (spot_move, scenario_name):
        - V_p_t: scenario put price
        - V_c_t: scenario call price
        - pnl_dollars: PnL of the strategy in dollars
        - pnl_pct: PnL as a percentage of |net_premium|
    """

    # -------------------------------------------- Unpack initial setup at T0 ----------------------------------------------
    initial_spot_price = initial_set_up["spot_price"]
    n_put_contracts    = initial_set_up["put_contracts"]
    n_call_contracts   = initial_set_up["call_contracts"]
    put_strike         = initial_set_up["put_strike"]
    call_strike        = initial_set_up["call_strike"]
    put_iv             = initial_set_up["put_iv"]
    call_iv            = initial_set_up["call_iv"]
    net_premium        = initial_set_up["net_premium"]
    risk_free_rate     = initial_set_up["risk_free_rate"]
    dividend_yield     = initial_set_up["dividend_yield"]
    binomial_steps     = initial_set_up["binomial_steps"]

    pricing_model_used = pricing_model_used.lower()

    # Pick initial option prices based on chosen model
    if pricing_model_used == "bs":
        V_p_0 = initial_set_up["put_price_bs"]
        V_c_0 = initial_set_up["call_price_bs"]
    elif pricing_model_used == "binomial":
        V_p_0 = initial_set_up["put_price_binom"]
        V_c_0 = initial_set_up["call_price_binom"]
    else:
        raise ValueError("pricing_model_used must be 'bs' or 'binomial'")
    
    # -------------------------------------------- Calculate Pnl for each scenario ----------------------------------------------

    # --- Container for results ---
    scenario_simulation_result: dict = {}

    # --- Loop over spot and IV scenarios ---
    for spot_move, scenario_dict in iv_scenarios.items():
        # Ensure nested dict exists
        scenario_simulation_result[spot_move*100] = {}

        # New spot is a percentage move from initial
        new_spot = initial_spot_price * (1 + spot_move)

        for scenario_name, iv_changes in scenario_dict.items():
            call_change = iv_changes["call_iv_change"]
            put_change  = iv_changes["put_iv_change"]

            # Apply IV changes
            new_put_iv  = put_iv  * (1 + put_change)
            new_call_iv = call_iv * (1 + call_change)


            print("\n--- SPOT MOVE ---")
            print(f"Before spot: {initial_spot_price}")
            print(f"Change : {spot_move * 100}%")
            print(f"After  spot: {new_spot}")

            print("\n--- IMPLIED VOLATILITY CHANGES ---")
            print("PUT IV:")
            print(f"  Before: {put_iv}")
            print(f"  Change: {put_change * 100:+}%")
            print(f"  After : {new_put_iv}")

            print("\nCALL IV:")
            print(f"  Before: {call_iv}")
            print(f"  Change: {call_change * 100:+}%")
            print(f"  After : {new_call_iv}")


            # Build a pricing model at the new spot
            pricing_model = model(
                spot_price=new_spot,
                risk_free_rate=risk_free_rate,
                dividend_yield=dividend_yield,
            )

            # Price options under chosen model
            if pricing_model_used == "bs":
                V_p_t = pricing_model.black_scholes_price(
                    K=put_strike,
                    T=time_to_maturity,
                    sigma=new_put_iv,
                    option_type="put",
                )
                V_c_t = pricing_model.black_scholes_price(
                    K=call_strike,
                    T=time_to_maturity,
                    sigma=new_call_iv,
                    option_type="call",
                )

            elif pricing_model_used == "binomial":
                V_p_t = pricing_model.crr_binomial_price(
                    K=put_strike,
                    T=time_to_maturity,
                    sigma=new_put_iv,
                    option_type="put",
                    steps=binomial_steps,
                    american=True,
                )
                V_c_t = pricing_model.crr_binomial_price(
                    K=call_strike,
                    T=time_to_maturity,
                    sigma=new_call_iv,
                    option_type="call",
                    steps=binomial_steps,
                    american=True,
                )

            # Compute PnL of the strategy
            pnl = compute_trade_pnl(
                n_put_contracts=n_put_contracts,
                n_call_contracts=n_call_contracts,
                V_p_0=V_p_0,
                V_p_t=V_p_t,
                V_c_0=V_c_0,
                V_c_t=V_c_t,
                net_premium=net_premium,
                risk_free_rate=risk_free_rate,
                rf_compunding_period=rf_compunding_period,
            )

            scenario_simulation_result[spot_move*100][scenario_name] = {
                "S_t": round(new_spot, 2),

                "put_iv_0":  round(put_iv,4),
                "Change_in_put_iv (%)":  round((new_put_iv / put_iv - 1) * 100, 4) if put_iv  != 0 else None,
                "put_iv_t":  round(new_put_iv,4),

                "call_iv_0": round(call_iv,4),
                "Change_in_call_iv (%)": round((new_call_iv / call_iv - 1) * 100, 4) if call_iv != 0 else None,
                "call_iv_t": round(new_call_iv,4),

                "V_p_0": round(V_p_0,4),
                "Change_in_put_price (%)":  round((V_p_t / V_p_0 - 1) * 100,4) if V_p_0 != 0 else None,
                "V_p_t": round(V_p_t,4),

                "V_c_0": round(V_c_0,4),
                "Change_in_call_price (%)": round((V_c_t / V_c_0 - 1) * 100,4) if V_c_0 != 0 else None,
                "V_c_t": round(V_c_t,4),

                "pnl_dollars": round(pnl,2),
            }


    return scenario_simulation_result

def simulation_result_to_df(scenario_simulation_result):
    """
    Convert a nested dictionary of the form:
        { move: { scenario: { field: value, ... } } }
    into a MultiIndex pandas DataFrame.
    
    Index levels:
        level 0 → move
        level 1 → scenario
    """
    
    # Flatten nested structure into tuple keys
    flat_dict = {
        (move, scenario): metrics
        for move, scenarios in scenario_simulation_result.items()
        for scenario, metrics in scenarios.items()
    }

    # Build DataFrame
    df = pd.DataFrame.from_dict(flat_dict, orient="index")

    # Set MultiIndex with names
    df.index = pd.MultiIndex.from_tuples(df.index, names=["Spot Move(%)", "scenario"])

    return df


In [12]:
scenario_simulation_result = scenario_simulation(
    initial_set_up,
    iv_scenarios,
    OptionsPricingModel,
    pricing_model_used ='binomial',
    time_to_maturity=7/365,
    rf_compunding_period=23/365
)

scenario_simulation_result 


--- SPOT MOVE ---
Before spot: 543.37
Change : 10.0%
After  spot: 597.7070000000001

--- IMPLIED VOLATILITY CHANGES ---
PUT IV:
  Before: 0.323284
  Change: -20.0%
  After : 0.2586272

CALL IV:
  Before: 0.210945
  Change: -8.0%
  After : 0.1940694

--- SPOT MOVE ---
Before spot: 543.37
Change : 10.0%
After  spot: 597.7070000000001

--- IMPLIED VOLATILITY CHANGES ---
PUT IV:
  Before: 0.323284
  Change: -30.0%
  After : 0.2262988

CALL IV:
  Before: 0.210945
  Change: -12.0%
  After : 0.1856316

--- SPOT MOVE ---
Before spot: 543.37
Change : 5.0%
After  spot: 570.5385

--- IMPLIED VOLATILITY CHANGES ---
PUT IV:
  Before: 0.323284
  Change: -12.0%
  After : 0.28448992

CALL IV:
  Before: 0.210945
  Change: -5.0%
  After : 0.20039774999999999

--- SPOT MOVE ---
Before spot: 543.37
Change : 5.0%
After  spot: 570.5385

--- IMPLIED VOLATILITY CHANGES ---
PUT IV:
  Before: 0.323284
  Change: -18.0%
  After : 0.26509288000000003

CALL IV:
  Before: 0.210945
  Change: -8.0%
  After : 0.194069

{10.0: {'moderate': {'S_t': np.float64(597.71),
   'put_iv_0': np.float64(0.3233),
   'Change_in_put_iv (%)': np.float64(-20.0),
   'put_iv_t': np.float64(0.2586),
   'call_iv_0': np.float64(0.2109),
   'Change_in_call_iv (%)': np.float64(-8.0),
   'call_iv_t': np.float64(0.1941),
   'V_p_0': np.float64(8.4911),
   'Change_in_put_price (%)': np.float64(-99.9991),
   'V_p_t': np.float64(0.0001),
   'V_c_0': np.float64(4.1914),
   'Change_in_call_price (%)': np.float64(551.4899),
   'V_c_t': np.float64(27.3065),
   'pnl_dollars': np.float64(5529905.46)},
  'strong': {'S_t': np.float64(597.71),
   'put_iv_0': np.float64(0.3233),
   'Change_in_put_iv (%)': np.float64(-30.0),
   'put_iv_t': np.float64(0.2263),
   'call_iv_0': np.float64(0.2109),
   'Change_in_call_iv (%)': np.float64(-12.0),
   'call_iv_t': np.float64(0.1856),
   'V_p_0': np.float64(8.4911),
   'Change_in_put_price (%)': np.float64(-100.0),
   'V_p_t': np.float64(0.0),
   'V_c_0': np.float64(4.1914),
   'Change_in_call_pric

In [13]:
df_scenario_analysis_result = simulation_result_to_df(scenario_simulation_result)
df_scenario_analysis_result.to_csv('../data/Result/1_Scenario_Analysis_Result.csv')
df_scenario_analysis_result

S_t  put_iv_0  Change_in_put_iv (%)  put_iv_t  \
Spot Move(%) scenario                                                     
 10.0        moderate  597.71    0.3233                 -20.0    0.2586   
             strong    597.71    0.3233                 -30.0    0.2263   
 5.0         moderate  570.54    0.3233                 -12.0    0.2845   
             strong    570.54    0.3233                 -18.0    0.2651   
 2.5         moderate  556.95    0.3233                  -5.0    0.3071   
             strong    556.95    0.3233                  -8.0    0.2974   
 0.0         moderate  543.37    0.3233                  -1.0    0.3201   
             strong    543.37    0.3233                  -2.0    0.3168   
-2.5         moderate  529.79    0.3233                   5.0    0.3394   
             strong    529.79    0.3233                  10.0    0.3556   
-5.0         moderate  516.20    0.3233                  15.0    0.3718   
             strong    516.20    0.3233                  25.0    0.4041   
-10.0        moderate  489.03    0.3233                  35.0    0.4364   
             strong    489.03    0.3233                  55.0    0.5011   

                       call_iv_0  Change_in_call_iv (%)  call_iv_t   V_p_0  \
Spot Move(%) scenario                                                        
 10.0        moderate     0.2109                   -8.0     0.1941  8.4911   
             strong       0.2109                  -12.0     0.1856  8.4911   
 5.0         moderate     0.2109                   -5.0     0.2004  8.4911   
             strong       0.2109                   -8.0     0.1941  8.4911   
 2.5         moderate     0.2109                   -2.0     0.2067  8.4911   
             strong       0.2109                   -4.0     0.2025  8.4911   
 0.0         moderate     0.2109                   -1.0     0.2088  8.4911   
             strong       0.2109                   -3.0     0.2046  8.4911   
-2.5         moderate     0.2109                    1.0     0.2131  8.4911   
             strong       0.2109                    3.0     0.2173  8.4911   
-5.0         moderate     0.2109                    5.0     0.2215  8.4911   
             strong       0.2109                    8.0     0.2278  8.4911   
-10.0        moderate     0.2109                   10.0     0.2320  8.4911   
             strong       0.2109                   15.0     0.2426  8.4911   

                       Change_in_put_price (%)    V_p_t   V_c_0  \
Spot Move(%) scenario                                             
 10.0        moderate                 -99.9991   0.0001  4.1914   
             strong                  -100.0000   0.0000  4.1914   
 5.0         moderate                 -99.5897   0.0348  4.1914   
             strong                   -99.7920   0.0177  4.1914   
 2.5         moderate                 -96.2579   0.3177  4.1914   
             strong                   -96.8831   0.2647  4.1914   
 0.0         moderate                 -83.8105   1.3747  4.1914   
             strong                   -84.3635   1.3277  4.1914   
-2.5         moderate                 -48.9982   4.3306  4.1914   
             strong                   -44.2714   4.7320  4.1914   
-5.0         moderate                  21.9932  10.3585  4.1914   
             strong                    32.8338  11.2790  4.1914   
-10.0        moderate                 251.8947  29.8797  4.1914   
             strong                   266.9192  31.1554  4.1914   

                       Change_in_call_price (%)    V_c_t  pnl_dollars  
Spot Move(%) scenario                                                  
 10.0        moderate                  551.4899  27.3065   5529905.46  
             strong                    550.1134  27.2488   5518229.68  
 5.0         moderate                   49.2151   6.2542   1263341.53  
             strong                     44.4563   6.0547   1224668.52  
 2.5         moderate                  -57.6203   1.7763    328278.85  